Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [15]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    sets = pickle.load(f)
    save = sets['sanitized']
    train_dataset = save[2][1]
    train_labels = save[2][2]
    valid_dataset = save[1][1]
    valid_labels = save[1][2]
    test_dataset = save[0][1]
    test_labels = save[0][2]
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (195106, 28, 28) (195106,)
Validation set (9840, 28, 28) (9840,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [16]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195106, 784) (195106, 10)
Validation set (9840, 784) (9840, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [18]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [23]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.565443
Training accuracy: 10.0%
Validation accuracy: 12.8%
Test accuracy: 13.1%
Loss at step 100: 2.423782
Training accuracy: 71.4%
Validation accuracy: 70.6%
Test accuracy: 78.6%
Loss at step 200: 1.921120
Training accuracy: 74.4%
Validation accuracy: 72.8%
Test accuracy: 80.9%
Loss at step 300: 1.653330
Training accuracy: 75.8%
Validation accuracy: 73.6%
Test accuracy: 81.7%
Loss at step 400: 1.478615
Training accuracy: 76.7%
Validation accuracy: 74.1%
Test accuracy: 82.2%
Loss at step 500: 1.351928
Training accuracy: 77.4%
Validation accuracy: 74.3%
Test accuracy: 82.5%
Loss at step 600: 1.253530
Training accuracy: 78.0%
Validation accuracy: 74.5%
Test accuracy: 82.6%
Loss at step 700: 1.173856
Training accuracy: 78.5%
Validation accuracy: 74.7%
Test accuracy: 82.7%
Loss at step 800: 1.107614
Training accuracy: 79.1%
Validation accuracy: 74.9%
Test accuracy: 82.8%


dfet's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [24]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.616913
Minibatch accuracy: 9.4%
Validation accuracy: 13.4%
Minibatch loss at step 500: 1.925560
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 1.108832
Minibatch accuracy: 81.2%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 1.350809
Minibatch accuracy: 75.8%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.878085
Minibatch accuracy: 79.7%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.714753
Minibatch accuracy: 78.1%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.791844
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Test accuracy: 86.4%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [64]:
batch_size = 128
num_relus = 1024

def make_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def make_bias(shape, constant=0.1):
    return tf.Variable(tf.constant(constant, shape=shape))

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    
    step1_weights = make_weights([image_size * image_size, num_relus])
    step1_biases = make_bias([num_relus])
    
    step2_weights = make_weights([num_relus, num_labels])
    step2_biases = make_bias([num_labels])
    
    # Training computation.
    def run_on_data(data):
        step1_logits = tf.matmul(data, step1_weights) + step1_biases
        intermediate_step = tf.nn.relu(step1_logits)
        my_logits = tf.matmul(intermediate_step, step2_weights) + step2_biases
        return my_logits
    
    logits = run_on_data(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(run_on_data(tf_valid_dataset))
    test_prediction = tf.nn.softmax(run_on_data(tf_test_dataset))


In [77]:
from timeit import default_timer as timer

num_steps = 10001
num_prints = 25

with tf.Session(graph=graph) as session:
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % (num_steps / num_prints) == 0):
            print("_____________________________")
            print("STEP %d" % step)
            print("loss %f" % l)
            end = timer()
            print("time to run last %d steps: %f" % (num_steps / num_prints, end - start))
            start = timer()
            print("Minibatch accuracy  : %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy : %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy       : %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
_____________________________
STEP 0
loss 4.338041
time to run last 400 steps: 0.185725
Minibatch accuracy  : 10.2%
Validation accuracy : 33.8%
Test accuracy       : 37.1%
_____________________________
STEP 400
loss 0.554116
time to run last 400 steps: 3.765518
Minibatch accuracy  : 86.7%
Validation accuracy : 84.3%
Test accuracy       : 90.9%
_____________________________
STEP 800
loss 0.421553
time to run last 400 steps: 3.769849
Minibatch accuracy  : 87.5%
Validation accuracy : 85.1%
Test accuracy       : 89.9%
_____________________________
STEP 1200
loss 0.398653
time to run last 400 steps: 4.048093
Minibatch accuracy  : 89.8%
Validation accuracy : 86.2%
Test accuracy       : 91.5%
_____________________________
STEP 1600
loss 0.551255
time to run last 400 steps: 3.951211
Minibatch accuracy  : 84.4%
Validation accuracy : 86.9%
Test accuracy       : 93.3%
_____________________________
STEP 2000
loss 0.322360
time to run last 400 steps: 3.973101
Minibatch accuracy  : 89.1%